In [4]:
import requests
import pandas as pd
import json
import base64
import os
from datetime import datetime

# El resto del código se mantiene igual
# Configuración de VALD
CLIENT_ID = "Nr37673W7ncT4qBQ=="
CLIENT_SECRET = "EMiYKddb7wKrwgxxLqopECDpkNLiS1XOEaw="
# TENANT_ID = "f1185650-fb79-44a0-8b4b-b2bf82d28c83"
FECHA_DESDE = "2023-01-01T00:00:00.000Z"






token = ''
# URL para solicitar el token
token_url = "https://security.valdperformance.com/connect/token"

# Datos para la solicitud del token
payload = {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET
}

try:
    # Realizar la solicitud
    response = requests.post(token_url, data=payload)
    
    # Verificar respuesta
    if response.status_code == 200:
        token_data = response.json()
        print(token_data)
        print("✅ Autenticación exitosa")
        token = token_data.get('access_token')
    else:
        print(f"❌ Error en la autenticación: {response.status_code}")
        print(response.text)
        
        
except Exception as e:
    print(f"❌ Error inesperado: {str(e)}")
   
   
   
tenant_id=''
# URL CORRECTA (la misma del cURL que funciona)
url = "https://prd-aue-api-externaltenants.valdperformance.com/tenants"

# Headers exactamente como en el cURL
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

try:

    response = requests.get(url, headers=headers)





def get_categories(tenant_id,token):
    # URL base y endpoint
    url = "https://prd-aue-api-externaltenants.valdperformance.com/categories"

    # Headers con token
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"
    }

    # Parámetros
    params = {
        "TenantId": tenant_id
    }

    # Realizar la solicitud
    print("🔄 Solicitando categories a la API...")
    response_categories = requests.get(url, headers=headers, params=params)
    
    # Decodificar y parsear JSON
    categories = json.loads(response_categories.content.decode('utf-8'))

    # 2. Crear DataFrame directamente
    df_categories_ = pd.DataFrame(categories['categories'])

    df_categories_['tenant_id'] = tenant_id

    return df_categories_





def get_groups(tenant_id,token):
    # URL base y endpoint
    url = "https://prd-aue-api-externaltenants.valdperformance.com/groups"

    # Headers con token
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"
    }

    # Parámetros
    params = {
        "TenantId": tenant_id
    }

    # Realizar la solicitud
    print("🔄 Solicitando grupos a la API...")
    response_groups = requests.get(url, headers=headers, params=params)
    
    # Decodificar y parsear JSON
    groups = json.loads(response_groups.content.decode('utf-8'))

    # 2. Crear DataFrame directamente
    df_groups_ = pd.DataFrame(groups['groups'])

    df_groups_['tenant_id'] = tenant_id

    return df_groups_







# Suponiendo que tus DataFrames se llaman df_categories y df_groups
# Hacer el merge usando el id de categoría
df_groups_with_category = df_groups.merge(
    df_categories[['id', 'name']], 
    left_on='categoryId', 
    right_on='id', 
    how='left'
)

# Renombrar la columna 'name' que viene de categorías para que sea más clara
df_groups_with_category = df_groups_with_category.rename(columns={'name_y': 'category_name', 'name_x': 'group_name'})

# O si prefieres ser más específico desde el merge:
df_groups_with_category = df_groups.merge(
    df_categories[['id', 'name']].rename(columns={'name': 'category_name'}), 
    left_on='categoryId', 
    right_on='id', 
    how='left'
)

# Eliminar la columna 'id' duplicada del merge
df_groups_with_category = df_groups_with_category.drop(columns=['id_y'])
df_groups_with_category = df_groups_with_category.rename(columns={'id_x': 'id'})
df_groups_with_category






def get_profiles(token, tenant_id, groupId, groupName, categoryId, categoryName):
    # URL base y endpoint
    url = "https://prd-aue-api-externalprofile.valdperformance.com/profiles"

    # Headers con token
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"
    }

    # Parámetros
    params = {
        "TenantId": tenant_id,
        "groupId": groupId
    }

    try:
        # Realizar la solicitud
        print("🔄 Solicitando perfiles a la API...")
        response_profiles = requests.get(url, headers=headers, params=params)
        
        # Verificar el status code
        if response_profiles.status_code != 200:
            print(f"⚠️ Error HTTP {response_profiles.status_code} para grupo {groupName}")
            return pd.DataFrame()  # Retorna DataFrame vacío
        
        # Verificar si hay contenido
        if not response_profiles.content or response_profiles.content == b'':
            print(f"⚠️ Respuesta vacía para grupo {groupName}")
            return pd.DataFrame()  # Retorna DataFrame vacío
        
        # Decodificar y parsear JSON
        content_str = response_profiles.content.decode('utf-8')
        
        # Verificar que no sea una cadena vacía
        if not content_str.strip():
            print(f"⚠️ Contenido vacío para grupo {groupName}")
            return pd.DataFrame()
            
        profiles = json.loads(content_str)
        
        # Verificar que tenga la estructura esperada
        if 'profiles' not in profiles:
            print(f"⚠️ Estructura JSON inesperada para grupo {groupName}")
            return pd.DataFrame()
        
        # Verificar que profiles no esté vacío
        if not profiles['profiles']:
            print(f"ℹ️ No hay perfiles para el grupo {groupName}")
            return pd.DataFrame()
        
        # Crear DataFrame
        df_profiles_ = pd.DataFrame(profiles['profiles'])
        
        # Agregar información del grupo
        df_profiles_['tenant_id'] = tenant_id
        df_profiles_['groupId'] = groupId
        df_profiles_['groupName'] = groupName
        df_profiles_['categoryId'] = categoryId
        df_profiles_['categoryName'] = categoryName
        
        print(f"✅ {len(df_profiles_)} perfiles obtenidos para {groupName}")
        return df_profiles_
        
    except json.JSONDecodeError as e:
        print(f"❌ Error JSON para grupo {groupName}: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"❌ Error inesperado para grupo {groupName}: {e}")
        return pd.DataFrame()
    
    
    
    
    
    
    
    
    # Recorrer df_groups_with_category
df_profiles = pd.DataFrame()
successful_requests = 0
failed_requests = 0

for index, row in df_groups_with_category.iterrows():
    print(f"\nProcesando grupo {index+1}/{len(df_groups_with_category)}: {row['id']}, {row['name']}...")
    
    df_profiles_ = get_profiles(token, row['tenant_id'], row['id'], 
                              row['name'], row['categoryId'], row['category_name'])
    
    if not df_profiles_.empty:
        df_profiles = pd.concat([df_profiles, df_profiles_], ignore_index=True)
        successful_requests += 1
    else:
        failed_requests += 1
        
        
        
        
        
        
        
        
        
        
        def get_nordbord(token, tenant_id, fecha_desde):
    """Obtiene los datos de pruebas desde la API de VALD"""
    
    # URL base según región
    base_url = "https://prd-aue-api-externalnordbord.valdperformance.com"
    
    # Endpoint y parámetros
    endpoint = "/tests/v2"
    params = {
        "tenantId": tenant_id,
        "modifiedFromUtc": fecha_desde
    }
    
    # Headers con token
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    # Realizar la solicitud
    print("🔄 Solicitando datos a la API...")
    response = requests.get(f"{base_url}{endpoint}", params=params, headers=headers)
    
    # Verificar respuesta
    if response.status_code == 200:
        print(f"✅ Datos obtenidos correctamente")
        datos = response.json()
    
        # Verificar tipo de datos y normalizarlos
        if isinstance(datos, dict) and 'items' in datos:
            df = pd.DataFrame(datos['items'])
        elif isinstance(datos, list):
            df = pd.DataFrame(datos)
        else:
            df = pd.DataFrame([datos])
        
        # Si existe la columna 'tests' y contiene datos, normalizarla
        if 'tests' in df.columns and len(df) > 0 and isinstance(df['tests'].iloc[0], (list, dict)):
            try:
                # Normalizar la columna tests
                expanded_df = pd.json_normalize(df['tests'].iloc[0])
                
                # Convertir columnas de fecha a datetime
                date_columns = [col for col in expanded_df.columns 
                                if 'date' in col.lower() or 'time' in col.lower()]
                for col in date_columns:
                    expanded_df[col] = pd.to_datetime(expanded_df[col], errors='ignore')
                
                # Reemplazar el DataFrame original con el normalizado
                df = expanded_df
                print("✅ Normalización aplicada a la columna 'tests'")
            except Exception as e:
                print(f"⚠️ No se pudo normalizar la columna 'tests': {e}")
        return df
    elif response.status_code == 204:
        print("⚠️ No hay más registros para obtener.")
        return []
    else:
        print(f"❌ Error al obtener datos: {response.status_code}")
        print(response.text)
        return None








# Recorrer df_groups_with_category
#get_nordbord(token, tenant_id, fecha_desde):
df_tests_nordbord = pd.DataFrame()
successful_requests = 0
failed_requests = 0
desde = "2023-01-01T00:00:00.000Z"

for index, row in df_tenants.iterrows():
    
    df_tests_nordbord_ = get_nordbord(token, row['id'], desde)
    
    if not df_tests_nordbord_.empty:
        df_tests_nordbord = pd.concat([df_tests_nordbord, df_tests_nordbord_], ignore_index=True)
        successful_requests += 1
    else:
        failed_requests += 1
   
   
   
   
   
   
   
   
   
def get_ForceFrame(token, tenant_id, fecha_desde):
    """Obtiene los datos de pruebas desde la API de VALD"""
    
    # URL base según región
    base_url = "https://prd-aue-api-externalforceframe.valdperformance.com/"
    
    # Endpoint y parámetros
    endpoint = "/tests/v2"
    params = {
        "tenantId": tenant_id,
        "modifiedFromUtc": fecha_desde
    }
    
    # Headers con token
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    # Realizar la solicitud
    print("🔄 Solicitando datos a la API...")
    response = requests.get(f"{base_url}{endpoint}", params=params, headers=headers)
    
    # Verificar respuesta
    if response.status_code == 200:
        print(f"✅ Datos obtenidos correctamente")
        datos = response.json()
    
        # Verificar tipo de datos y normalizarlos
        if isinstance(datos, dict) and 'items' in datos:
            df = pd.DataFrame(datos['items'])
        elif isinstance(datos, list):
            df = pd.DataFrame(datos)
        else:
            df = pd.DataFrame([datos])
        
        # Si existe la columna 'tests' y contiene datos, normalizarla
        if 'tests' in df.columns and len(df) > 0 and isinstance(df['tests'].iloc[0], (list, dict)):
            try:
                # Normalizar la columna tests
                expanded_df = pd.json_normalize(df['tests'].iloc[0])
                
                # Convertir columnas de fecha a datetime
                date_columns = [col for col in expanded_df.columns 
                                if 'date' in col.lower() or 'time' in col.lower()]
                for col in date_columns:
                    expanded_df[col] = pd.to_datetime(expanded_df[col], errors='ignore')
                
                # Reemplazar el DataFrame original con el normalizado
                df = expanded_df
                print("✅ Normalización aplicada a la columna 'tests'")
            except Exception as e:
                print(f"⚠️ No se pudo normalizar la columna 'tests': {e}")
        return df
    elif response.status_code == 204:
        print("⚠️ No hay más registros para obtener.")
        return []
    else:
        print(f"❌ Error al obtener datos: {response.status_code}")
        print(response.text)
        return None



df_tests_ForceFrame = pd.DataFrame()
successful_requests = 0
failed_requests = 0
desde = "2023-01-01T00:00:00.000Z"

for index, row in df_tenants.iterrows():
    
    df_tests_ForceFrame_ = get_ForceFrame(token, row['id'], desde)

    if not df_tests_ForceFrame_.empty:
        df_tests_ForceFrame = pd.concat([df_tests_ForceFrame, df_tests_ForceFrame_], ignore_index=True)
        successful_requests += 1
    else:
        failed_requests += 1

SyntaxError: expected 'except' or 'finally' block (2220622611.py, line 69)

In [6]:
import requests
import pandas as pd
import json
import base64
import os
from datetime import datetime

# Configuración de VALD
CLIENT_ID = "Nr37673W7ncT4qBQ=="
CLIENT_SECRET = "EMiYKddb7wKrwgxxLqopECDpkNLiS1XOEaw="
FECHA_DESDE = "2023-01-01T00:00:00.000Z"

# Crear directorio para guardar los CSV si no existe
OUTPUT_DIR = "output_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Función para obtener token
def get_token():
    token_url = "https://security.valdperformance.com/connect/token"
    
    payload = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET
    }
    
    try:
        response = requests.post(token_url, data=payload)
        
        if response.status_code == 200:
            token_data = response.json()
            print("✅ Autenticación exitosa")
            return token_data.get('access_token')
        else:
            print(f"❌ Error en la autenticación: {response.status_code}")
            print(response.text)
            return None
            
    except Exception as e:
        print(f"❌ Error inesperado: {str(e)}")
        return None

# Función para obtener tenants
def get_tenants(token):
    url = "https://prd-aue-api-externaltenants.valdperformance.com/tenants"
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"
    }
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            tenants_data = response.json()
            df_tenants = pd.DataFrame(tenants_data['tenants'])
            
            # Guardar a CSV
            csv_path = os.path.join(OUTPUT_DIR, "tenants.csv")
            df_tenants.to_csv(csv_path, index=False)
            print(f"✅ Datos de tenants guardados en {csv_path}")
            
            return df_tenants
        else:
            print(f"❌ Error al obtener tenants: {response.status_code}")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"❌ Error inesperado: {str(e)}")
        return pd.DataFrame()

# Función para obtener categorías
def get_categories(tenant_id, token):
    url = "https://prd-aue-api-externaltenants.valdperformance.com/categories"
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"
    }
    
    params = {
        "TenantId": tenant_id
    }
    
    try:
        print("🔄 Solicitando categories a la API...")
        response_categories = requests.get(url, headers=headers, params=params)
        
        if response_categories.status_code != 200:
            print(f"❌ Error al obtener categorías: {response_categories.status_code}")
            return pd.DataFrame()
            
        categories = json.loads(response_categories.content.decode('utf-8'))
        df_categories_ = pd.DataFrame(categories['categories'])
        df_categories_['tenant_id'] = tenant_id
        
        # Guardar a CSV
        csv_path = os.path.join(OUTPUT_DIR, f"categories_{tenant_id}.csv")
        df_categories_.to_csv(csv_path, index=False)
        print(f"✅ Categorías para tenant {tenant_id} guardadas en {csv_path}")
        
        return df_categories_
        
    except Exception as e:
        print(f"❌ Error al obtener categorías: {str(e)}")
        return pd.DataFrame()

# Función para obtener grupos
def get_groups(tenant_id, token):
    url = "https://prd-aue-api-externaltenants.valdperformance.com/groups"
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"
    }
    
    params = {
        "TenantId": tenant_id
    }
    
    try:
        print("🔄 Solicitando grupos a la API...")
        response_groups = requests.get(url, headers=headers, params=params)
        
        if response_groups.status_code != 200:
            print(f"❌ Error al obtener grupos: {response_groups.status_code}")
            return pd.DataFrame()
            
        groups = json.loads(response_groups.content.decode('utf-8'))
        df_groups_ = pd.DataFrame(groups['groups'])
        df_groups_['tenant_id'] = tenant_id
        
        # Guardar a CSV
        csv_path = os.path.join(OUTPUT_DIR, f"groups_{tenant_id}.csv")
        df_groups_.to_csv(csv_path, index=False)
        print(f"✅ Grupos para tenant {tenant_id} guardados en {csv_path}")
        
        return df_groups_
        
    except Exception as e:
        print(f"❌ Error al obtener grupos: {str(e)}")
        return pd.DataFrame()

# Función para obtener perfiles
# Función para obtener perfiles (modificada para un solo CSV)
def get_profiles(token, tenant_id, groupId, groupName, categoryId, categoryName, df_all_profiles=None):
    url = "https://prd-aue-api-externalprofile.valdperformance.com/profiles"
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"
    }
    
    params = {
        "TenantId": tenant_id,
        "groupId": groupId
    }
    
    try:
        print(f"🔄 Solicitando perfiles para grupo {groupName}...")
        response_profiles = requests.get(url, headers=headers, params=params)
        
        if response_profiles.status_code != 200:
            print(f"⚠️ Error HTTP {response_profiles.status_code} para grupo {groupName}")
            return df_all_profiles if df_all_profiles is not None else pd.DataFrame()
        
        if not response_profiles.content or response_profiles.content == b'':
            print(f"⚠️ Respuesta vacía para grupo {groupName}")
            return df_all_profiles if df_all_profiles is not None else pd.DataFrame()
        
        content_str = response_profiles.content.decode('utf-8')
        
        if not content_str.strip():
            print(f"⚠️ Contenido vacío para grupo {groupName}")
            return df_all_profiles if df_all_profiles is not None else pd.DataFrame()
            
        profiles = json.loads(content_str)
        
        if 'profiles' not in profiles:
            print(f"⚠️ Estructura JSON inesperada para grupo {groupName}")
            return df_all_profiles if df_all_profiles is not None else pd.DataFrame()
        
        if not profiles['profiles']:
            print(f"ℹ️ No hay perfiles para el grupo {groupName}")
            return df_all_profiles if df_all_profiles is not None else pd.DataFrame()
        
        df_profiles_ = pd.DataFrame(profiles['profiles'])
        
        df_profiles_['tenant_id'] = tenant_id
        df_profiles_['groupId'] = groupId
        df_profiles_['groupName'] = groupName
        df_profiles_['categoryId'] = categoryId
        df_profiles_['categoryName'] = categoryName
        
        # Si ya tenemos un DataFrame de todos los perfiles, concatenamos
        if df_all_profiles is not None:
            df_all_profiles = pd.concat([df_all_profiles, df_profiles_], ignore_index=True)
        else:
            df_all_profiles = df_profiles_
        
        print(f"✅ {len(df_profiles_)} perfiles obtenidos para grupo {groupName}")
        
        return df_all_profiles
        
    except json.JSONDecodeError as e:
        print(f"❌ Error JSON para grupo {groupName}: {e}")
        return df_all_profiles if df_all_profiles is not None else pd.DataFrame()
    except Exception as e:
        print(f"❌ Error inesperado para grupo {groupName}: {e}")
        return df_all_profiles if df_all_profiles is not None else pd.DataFrame()
    

# Función para obtener datos de NordBord
def get_nordbord(token, tenant_id, fecha_desde):
    base_url = "https://prd-aue-api-externalnordbord.valdperformance.com"
    
    endpoint = "/tests/v2"
    params = {
        "tenantId": tenant_id,
        "modifiedFromUtc": fecha_desde
    }
    
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    try:
        print(f"🔄 Solicitando datos NordBord para tenant {tenant_id}...")
        response = requests.get(f"{base_url}{endpoint}", params=params, headers=headers)
        
        if response.status_code == 200:
            print(f"✅ Datos obtenidos correctamente")
            datos = response.json()
        
            if isinstance(datos, dict) and 'items' in datos:
                df = pd.DataFrame(datos['items'])
            elif isinstance(datos, list):
                df = pd.DataFrame(datos)
            else:
                df = pd.DataFrame([datos])
            
            if 'tests' in df.columns and len(df) > 0 and isinstance(df['tests'].iloc[0], (list, dict)):
                try:
                    expanded_df = pd.json_normalize(df['tests'].iloc[0])
                    
                    date_columns = [col for col in expanded_df.columns 
                                    if 'date' in col.lower() or 'time' in col.lower()]
                    for col in date_columns:
                        expanded_df[col] = pd.to_datetime(expanded_df[col], errors='ignore')
                    
                    df = expanded_df
                    print("✅ Normalización aplicada a la columna 'tests'")
                except Exception as e:
                    print(f"⚠️ No se pudo normalizar la columna 'tests': {e}")
            
            # Guardar a CSV
            if not df.empty:
                csv_path = os.path.join(OUTPUT_DIR, f"nordbord_{tenant_id}.csv")
                df.to_csv(csv_path, index=False)
                print(f"✅ Datos NordBord para tenant {tenant_id} guardados en {csv_path}")
            
            return df
            
        elif response.status_code == 204:
            print("⚠️ No hay más registros para obtener.")
            return pd.DataFrame()
        else:
            print(f"❌ Error al obtener datos: {response.status_code}")
            print(response.text)
            return pd.DataFrame()
            
    except Exception as e:
        print(f"❌ Error al obtener datos NordBord: {str(e)}")
        return pd.DataFrame()

# Función para obtener datos de ForceFrame
def get_ForceFrame(token, tenant_id, fecha_desde):
    base_url = "https://prd-aue-api-externalforceframe.valdperformance.com"
    
    endpoint = "/tests/v2"
    params = {
        "tenantId": tenant_id,
        "modifiedFromUtc": fecha_desde
    }
    
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    try:
        print(f"🔄 Solicitando datos ForceFrame para tenant {tenant_id}...")
        response = requests.get(f"{base_url}{endpoint}", params=params, headers=headers)
        
        if response.status_code == 200:
            print(f"✅ Datos obtenidos correctamente")
            datos = response.json()
        
            if isinstance(datos, dict) and 'items' in datos:
                df = pd.DataFrame(datos['items'])
            elif isinstance(datos, list):
                df = pd.DataFrame(datos)
            else:
                df = pd.DataFrame([datos])
            
            if 'tests' in df.columns and len(df) > 0 and isinstance(df['tests'].iloc[0], (list, dict)):
                try:
                    expanded_df = pd.json_normalize(df['tests'].iloc[0])
                    
                    date_columns = [col for col in expanded_df.columns 
                                    if 'date' in col.lower() or 'time' in col.lower()]
                    for col in date_columns:
                        expanded_df[col] = pd.to_datetime(expanded_df[col], errors='ignore')
                    
                    df = expanded_df
                    print("✅ Normalización aplicada a la columna 'tests'")
                except Exception as e:
                    print(f"⚠️ No se pudo normalizar la columna 'tests': {e}")
            
            # Guardar a CSV
            if not df.empty:
                csv_path = os.path.join(OUTPUT_DIR, f"forceframe_{tenant_id}.csv")
                df.to_csv(csv_path, index=False)
                print(f"✅ Datos ForceFrame para tenant {tenant_id} guardados en {csv_path}")
            
            return df
            
        elif response.status_code == 204:
            print("⚠️ No hay más registros para obtener.")
            return pd.DataFrame()
        else:
            print(f"❌ Error al obtener datos: {response.status_code}")
            print(response.text)
            return pd.DataFrame()
            
    except Exception as e:
        print(f"❌ Error al obtener datos ForceFrame: {str(e)}")
        return pd.DataFrame()

# Función principal para ejecutar todo el proceso
def run_extraction():
    # Obtener token
    token = get_token()
    if not token:
        print("❌ No se pudo obtener el token. Proceso cancelado.")
        return
    
    # Obtener tenants
    df_tenants = get_tenants(token)
    if df_tenants.empty:
        print("❌ No se pudieron obtener los tenants. Proceso cancelado.")
        return
    
    # Procesar cada tenant
    for index, tenant in df_tenants.iterrows():
        tenant_id = tenant['id']
        tenant_name = tenant['name']
        print(f"\n🔍 Procesando tenant {index+1}/{len(df_tenants)}: {tenant_name}")
        
        # Obtener categorías
        df_categories = get_categories(tenant_id, token)
        
        # Obtener grupos
        df_groups = get_groups(tenant_id, token)
        
        # Si hay categorías y grupos, combinarlos
        if not df_categories.empty and not df_groups.empty:
            # Combinar grupos con categorías
            df_groups_with_category = df_groups.merge(
                df_categories[['id', 'name']].rename(columns={'name': 'category_name'}),
                left_on='categoryId',
                right_on='id',
                how='left'
            )
            
            # Limpiar columnas duplicadas
            if 'id_y' in df_groups_with_category.columns:
                df_groups_with_category = df_groups_with_category.drop(columns=['id_y'])
                df_groups_with_category = df_groups_with_category.rename(columns={'id_x': 'id'})
            
            # Guardar grupos con categorías
            csv_path = os.path.join(OUTPUT_DIR, f"groups_with_categories_{tenant_id}.csv")
            df_groups_with_category.to_csv(csv_path, index=False)
            print(f"✅ Grupos con categorías guardados en {csv_path}")
            
            # Obtener perfiles por grupo
            print("\n📊 Obteniendo perfiles para cada grupo...")
            for idx, group in df_groups_with_category.iterrows():
                get_profiles(
                    token, 
                    tenant_id, 
                    group['id'], 
                    group['name'], 
                    group['categoryId'], 
                    group.get('category_name', '')
                )
        
        # Obtener datos de NordBord
        print("\n📊 Obteniendo datos de NordBord...")
        get_nordbord(token, tenant_id, FECHA_DESDE)
        
        # Obtener datos de ForceFrame
        print("\n📊 Obteniendo datos de ForceFrame...")
        get_ForceFrame(token, tenant_id, FECHA_DESDE)

# Ejecutar el proceso completo
if __name__ == "__main__":
    run_extraction()

✅ Autenticación exitosa
✅ Datos de tenants guardados en output_data\tenants.csv

🔍 Procesando tenant 1/1: STAFF - Luciano Tomagheli
🔄 Solicitando categories a la API...
✅ Categorías para tenant f1185650-fb79-44a0-8b4b-b2bf82d28c83 guardadas en output_data\categories_f1185650-fb79-44a0-8b4b-b2bf82d28c83.csv
🔄 Solicitando grupos a la API...
✅ Grupos para tenant f1185650-fb79-44a0-8b4b-b2bf82d28c83 guardados en output_data\groups_f1185650-fb79-44a0-8b4b-b2bf82d28c83.csv
✅ Grupos con categorías guardados en output_data\groups_with_categories_f1185650-fb79-44a0-8b4b-b2bf82d28c83.csv

📊 Obteniendo perfiles para cada grupo...
🔄 Solicitando perfiles para grupo 1...
✅ 30 perfiles obtenidos para grupo 1
🔄 Solicitando perfiles para grupo 2...
✅ 30 perfiles obtenidos para grupo 2
🔄 Solicitando perfiles para grupo 3...
✅ 28 perfiles obtenidos para grupo 3
🔄 Solicitando perfiles para grupo 4...
✅ 30 perfiles obtenidos para grupo 4
🔄 Solicitando perfiles para grupo 4ta...
✅ 39 perfiles obtenidos para

C:\Users\dell\AppData\Local\Temp\ipykernel_1088\2566829462.py:250: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  expanded_df[col] = pd.to_datetime(expanded_df[col], errors='ignore')
C:\Users\dell\AppData\Local\Temp\ipykernel_1088\2566829462.py:250: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  expanded_df[col] = pd.to_datetime(expanded_df[col], errors='ignore')


✅ Datos obtenidos correctamente
✅ Normalización aplicada a la columna 'tests'
✅ Datos ForceFrame para tenant f1185650-fb79-44a0-8b4b-b2bf82d28c83 guardados en output_data\forceframe_f1185650-fb79-44a0-8b4b-b2bf82d28c83.csv


C:\Users\dell\AppData\Local\Temp\ipykernel_1088\2566829462.py:313: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  expanded_df[col] = pd.to_datetime(expanded_df[col], errors='ignore')
C:\Users\dell\AppData\Local\Temp\ipykernel_1088\2566829462.py:313: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  expanded_df[col] = pd.to_datetime(expanded_df[col], errors='ignore')
